In [1]:
%%writefile app.py

import streamlit as st
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.callbacks.base import BaseCallbackHandler
import openai
import requests

# Set up Streamlit app
st.set_page_config(page_title="Arthlantis AI", page_icon="🤖")
st.title("Welcome to Arthlantis, I am AI Arthur! What can I help you with today? 🤖")

# Sidebar for settings
st.sidebar.header("Settings")

# Custom model names for display
model_display_names = {
    "Arthlantis 1": "gpt-3.5-turbo",
    "Arthlantis 2": "gpt-4"
}

# Model selection dropdown (users see custom names, backend uses real model names)
selected_display_name = st.sidebar.selectbox("Choose AI Version:", list(model_display_names.keys()), index=0)
selected_model = model_display_names[selected_display_name]

# Temperature slider
temperature = st.sidebar.slider("Temperature:", min_value=0.0, max_value=1.0, value=0.1, step=0.1)

# Button to clear chat
if st.sidebar.button("Clear Chat History"):
    st.session_state.messages = []
    st.experimental_rerun()

# File upload function
def handle_file_upload():
    uploaded_file = st.file_uploader("Upload a file", type=["txt", "pdf", "docx", "csv"])
    if uploaded_file is not None:
        if uploaded_file.type == "text/plain":
            text = uploaded_file.read().decode("utf-8")
            st.text_area("File Content", value=text, height=200)
        elif uploaded_file.type == "application/pdf":
            # Use PyPDF2 or pdfplumber to extract text
            import PyPDF2
            pdf_reader = PyPDF2.PdfReader(uploaded_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            st.text_area("PDF Content", value=text, height=200)
        elif uploaded_file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
            from docx import Document
            doc = Document(uploaded_file)
            text = ""
            for para in doc.paragraphs:
                text += para.text
            st.text_area("Docx Content", value=text, height=200)
        elif uploaded_file.type == "text/csv":
            import pandas as pd
            df = pd.read_csv(uploaded_file)
            st.write(df)
            st.text_area("CSV Content", value=df.to_string(), height=200)
    return uploaded_file

# Real-time web research functionality
def web_search(query):
    st.write(f"Searching the web for: {query}")
    response = requests.get(f"https://api.duckduckgo.com/?q={query}&format=json")
    if response.status_code == 200:
        results = response.json()
        try:
            abstract = results.get("AbstractText", "No relevant information found.")
            st.write(abstract)
        except KeyError:
            st.write("Could not retrieve web data.")
    else:
        st.write("Web search failed. Please try again.")

# StreamHandler for live updates
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)

# Load ChatGPT with selected settings
try:
    chatgpt = ChatOpenAI(model_name=selected_model, temperature=temperature, streaming=True)
except Exception as e:
    st.error(f"⚠️ Model Error: {e}")
    st.stop()

# System prompt for assistant personality
SYS_PROMPT = """
    Act as a helpful and humorous assistant. Be engaging, human-like, and ensure conversations are relaxed and enjoyable.
"""

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", SYS_PROMPT),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# LLM chain
llm_chain = prompt | chatgpt

# Conversation history
streamlit_msg_history = StreamlitChatMessageHistory()
conversation_chain = RunnableWithMessageHistory(
    llm_chain,
    lambda session_id: streamlit_msg_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Display initial message
if len(streamlit_msg_history.messages) == 0:
    streamlit_msg_history.add_ai_message("How can I assist you today?")

# Render past messages
for msg in streamlit_msg_history.messages:
    st.chat_message(msg.type).write(msg.content)

# Handle new user input
if user_prompt := st.chat_input():
    st.chat_message("human").write(user_prompt)

    if "search" in user_prompt.lower():
        web_search(user_prompt)
    else:
        with st.chat_message("ai"):
            stream_handler = StreamHandler(st.empty())
            config = {"configurable": {"session_id": "any"}, "callbacks": [stream_handler]}
            response = conversation_chain.invoke({"input": user_prompt}, config)

# File upload
handle_file_upload()


Writing app.py
